In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_ag_news(split='train'):
    data = pd.read_csv('data/ag_news_csv/{}.csv'.format(split),header=-1)
    labels = data[0].values.tolist()
    texts = data[1]+data[2]
    texts = texts.values.tolist()
    return texts,labels

In [3]:
texts,labels = load_ag_news()

In [4]:
def text_preprocessing(texts):
    #只保留字母和数字空格
    texts = [re.sub(r'[^A-Za-z0-9 ]','',text) for text in texts]
    #去空格字母小写
    texts = [[word.strip().lower() for word in text.split(' ')] for text in texts]
    #去停用词
    english_stopwords = stopwords.words('english')
    texts = [list(filter(lambda x:x not in english_stopwords,text)) for text in texts]
    #Stemming
    stemmer = SnowballStemmer('english')
    texts = [[stemmer.stem(word) for word in text] for text in texts]
    #Lemma
    lemma = WordNetLemmatizer()
    texts = [[lemma.lemmatize(word) for word in text] for text in texts]
    texts = [' '.join(text) for text in texts]
    return texts

In [5]:
texts=text_preprocessing(texts)
texts[0]

'wall st bear claw back black reutersreut  shortsel wall street dwindlingband ultracyn see green'

In [6]:
def preprocess_label_for_model(labels):
    onehot_labels = np_utils.to_categorical(labels)
    onehot_labels = onehot_labels[:,1:]
    return onehot_labels

In [7]:
onehot_labels = preprocess_label_for_model(labels)

In [8]:
x_train,x_val,y_train,y_val = train_test_split(texts,onehot_labels,test_size=0.001,shuffle=True,random_state=42)

In [9]:
def preprocess_text_for_model(x,token=None,dict_size=8000,max_length=128):
    if token is None:
        token = Tokenizer(num_words=dict_size)
        token.fit_on_texts(x)
    texts_seq = token.texts_to_sequences(x)
    texts_seq_pad = sequence.pad_sequences(texts_seq,maxlen=max_length)
    return texts_seq_pad,token

In [10]:
x_train_seq,token = preprocess_text_for_model(x_train)
x_val_seq,_ = preprocess_text_for_model(x_val,token)

In [11]:
dimension_output=4
dict_size=8000
embedded_size=128
maxlen=128
pooling_size=5
num_layers=2
size_layer=128
n_filters=250

In [12]:
tf.reset_default_graph()

In [13]:
x = tf.placeholder(tf.int32,shape=(None,maxlen),name='x')
y = tf.placeholder(tf.float32,shape=(None,dimension_output),name='y')

In [14]:
def kmax_pooling(x):
    x_ = tf.transpose(x,[0,2,1])
    x_ = tf.nn.top_k(x_,5,sorted=False).values
    x_ = tf.transpose(x_,[0,2,1])
    return tf.reshape(x_,[-1,5,n_filters//3])

In [15]:
with tf.name_scope('embedding'):
    embedding_matrix = tf.Variable(tf.random_normal([dict_size,embedded_size],-1,1))
    embedding = tf.nn.embedding_lookup(embedding_matrix,x)
with tf.name_scope('cnn'):
    parallels = []
    for kernel_size in [3,4,5]:
        conv = tf.layers.conv1d(embedding,filters=n_filters//3,kernel_size=kernel_size,activation='relu') #conv shape: batch_size * conv_output_size * filters
        pool = kmax_pooling(conv)
        parallels.append(pool)
with tf.name_scope('concat'):
    concat = tf.concat(parallels,axis=-1)
    parall = tf.reshape(concat,[-1,5*(n_filters//3)*3])
with tf.name_scope('output'):
    logits = tf.layers.dense(parall,dimension_output,name='logits')
    pred = tf.argmax(logits,1)
with tf.name_scope('train'):
    loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss_func)
with tf.name_scope('eval'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits,1),tf.argmax(y,1)),tf.float32))

In [16]:
def shuffle_batch(x,y,batch_size):
    index = np.random.permutation(len(x))
    n_batch = len(x) // batch_size
    for batch_index in np.array_split(index,n_batch):
        x_batch,y_batch = x[batch_index],y[batch_index]
        yield x_batch,y_batch

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [22]:
epochs = 5
batch_size = 30

In [23]:
for epoch in range(epochs):
    for x_batch,y_batch in shuffle_batch(x_train_seq,y_train,batch_size):
        sess.run(optimizer,feed_dict={x:x_batch,y:y_batch})
    l,a = sess.run([loss_func,accuracy],feed_dict={x:x_val_seq,y:y_val})
    print('epoch:%d loss:%.2f acc:%.2f' %(epoch,l,a))

epoch:0 loss:0.47 acc:0.87
epoch:1 loss:0.37 acc:0.86
epoch:2 loss:0.31 acc:0.89
epoch:3 loss:0.28 acc:0.93
epoch:4 loss:0.31 acc:0.91


In [24]:
sents_test,labels_test=load_ag_news('test')

In [25]:
texts_test=text_preprocessing(sents_test)

In [26]:
onehot_labels_test = preprocess_label_for_model(labels_test)
x_test_seq,_=preprocess_text_for_model(sents_test,token)

In [33]:
y_pred=[]
for batch in np.array_split(np.arange(x_test_seq.shape[0]),100):
    p = sess.run(pred,feed_dict={x:x_test_seq[batch]})
    y_pred.extend(p)

In [35]:
final_result = [item+1 for item in y_pred]

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(labels_test,final_result)

0.7785526315789474

In [37]:
sess.close()